In [1]:
import pandas as pd 

In [2]:
# Load the JSON file into a pandas DataFrame
df = pd.read_json('News_Category_Dataset_v3.json', lines=True)

In [3]:
df = df [['category',	'short_description']]
df.head()

,category,short_description
0,U.S. NEWS,Health experts said it is too early to predict...
1,U.S. NEWS,He was subdued by passengers and crew when he ...
2,COMEDY,"""Until you have a dog you don't understand wha..."
3,PARENTING,"""Accidentally put grown-up toothpaste on my to..."
4,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...


Create the Bag of Words using CountVectorizer from df['short_description']

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
# Creating Bag of Words
headlines = df['short_description']

# with stop‑words
cv_with = CountVectorizer()
bow_with  = cv_with.fit_transform(headlines)
vocab_size_with = len(cv_with.get_feature_names_out())

# without stop‑words
cv_no = CountVectorizer(stop_words='english')
bow_no = cv_no.fit_transform(headlines)
vocab_size_no = len(cv_no.get_feature_names_out())

# three most‑frequent tokens (stop‑words removed version)
word_counts = bow_no.sum(axis=0).A1         # flatten matrix to 1‑D
vocab       = cv_no.get_feature_names_out()
top_idx     = word_counts.argsort()[-3:][::-1]
top_words   = [(vocab[i], int(word_counts[i])) for i in top_idx]

print("BoW vocab size (WITH stop-words) :", vocab_size_with)
print("BoW vocab size (NO  stop-words) :", vocab_size_no)
print("Top 3 frequent words (stop-words removed) :", top_words)

BoW vocab size (WITH stop-words) : 75726
BoW vocab size (NO  stop-words) : 75420
Top 3 frequent words (stop-words removed) : [('new', 10730), ('people', 9801), ('time', 9791)]


In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(sublinear_tf=True, use_idf=True)

In [8]:
# Sample code. count_vect is the name of your Countvectorizor

test_sentence = "I do not understand this assignment"
X_tst = cv_with.transform([test_sentence])
X_tst.shape


(1, 75726)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Bag‑of‑Words counts
count_vect = CountVectorizer(stop_words='english')
X_counts   = count_vect.fit_transform(headlines)   # headlines is your df['short_description']

# TF‑IDF transform
tfidf_transformer = TfidfTransformer(sublinear_tf=True, use_idf=True)
X_tfidf = tfidf_transformer.fit_transform(X_counts)

# Example: transform a new sentence
test_sentence = "I do not understand this assignment"
X_tst_counts  = count_vect.transform([test_sentence])
tfidf_vec     = tfidf_transformer.transform(X_tst_counts)

print("TF-IDF shape for test sentence:", tfidf_vec.shape)  # (1, vocab_size)


TF‑IDF shape for test sentence: (1, 75420)


In [ ]:
'''
tfidf_vectors_test = tfidf_transformer.transform(X_tst)
tfidf_vectors_test
'''

ValueError: could not convert string to float: 'Health experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.'

In [15]:
## Q5) b
import spacy
nlp = spacy.load("en_core_web_sm")
sample_idx   = 6721
sample_text  = df.iloc[sample_idx]['short_description']
doc          = nlp(sample_text)

print(f"\nspaCy NER for row {sample_idx}:")
for ent in doc.ents:
    print(f"{ent.text:30}  →  {ent.label_}")


spaCy NER for row 6721:
Joe Perricone                   →  PERSON
95                              →  DATE
Bill William Arnold Craddock    →  PERSON
85                              →  DATE
more than 65 years              →  DATE


In [21]:
## Q5) c
from transformers import pipeline

hf_ner = pipeline("ner", grouped_entities=True)
hf_entities = hf_ner(sample_text)

print("\nTransformer NER:")
for ent in hf_entities:
    print(f"{ent['word']:30}  →  {ent['entity_group']} "
          f"(score={ent['score']:.3f})")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0



Transformer NER:
Joe Perricone                   →  PER (score=0.998)
Bill William Arnold Craddock    →  PER (score=0.967)


In [25]:
## Q5) c
from transformers import pipeline

hf_ner = pipeline("ner", grouped_entities=True)
hf_entities = hf_ner(sample_text)
(hf_entities)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


[{'entity_group': 'PER',
  'score': np.float32(0.9983566),
  'word': 'Joe Perricone',
  'start': 0,
  'end': 13},
 {'entity_group': 'PER',
  'score': np.float32(0.96669865),
  'word': 'Bill William Arnold Craddock',
  'start': 23,
  'end': 51}]

In [ ]:
## Q5) d
from transformers import pipeline
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

extractor = pipeline("feature-extraction", 
                     model="bert-base-uncased",
                     tokenizer="bert-base-uncased")

def sentence_embedding(text):
    """average the token vectors to get one vector per sentence"""
    token_vectors = np.array(extractor(text)[0])   # (tokens, 768)
    return token_vectors.mean(axis=0)              # (768,)

vec1 = sentence_embedding("I do not understand this assignment.")
vec2 = sentence_embedding("This assignment is pretty clear.")

print("\nEmbedding dimension:", vec1.shape[0])

emb_montreal = extractor("Montreal")
aa = np.array(emb_montreal)
print ('Embedding shape for Montreal: ',aa.shape)

sim = cosine_similarity(vec1.reshape(1,-1), vec2.reshape(1,-1))[0][0]
print("Cosine similarity between the two example sentences:", f"{sim:.3f}")


Device set to use mps:0



Embedding dimension: 768
embedding shape for Montreal:  (1, 3, 768)
Cosine similarity between the two example sentences: 0.725


In [29]:
emb_montreal = extractor("Montreal")
aa = np.array(emb_montreal)
print (aa.shape)

(1, 3, 768)
